# 📚 RAG 시스템 완전 정복: 을GPT 1주차 

을GPT ai및rag팀을 위한 노트북입니다.
공유를 금지합니다. 

## 🎯 학습 목표

이 노트북을 통해 다음을 배울 수 있습니다:

### 📖 이론적 이해
- ✅ RAG(Retrieval-Augmented Generation)의 핵심 개념
- ✅ 임베딩과 벡터 검색의 원리
- ✅ 프롬프트 엔지니어링 기법
- ✅ AI 시스템 평가 및 개선 방법

# 🤖 Google Gemini API를 활용한 RAG(Retrieval-Augmented Generation) 시스템 구축

## 📚 RAG란 무엇인가요?

**RAG(Retrieval-Augmented Generation)**는 대화형 AI의 새로운 패러다임입니다.

### 🔍 기존 AI와 RAG의 차이점

| 구분 | 기존 AI 모델 | RAG 시스템 |
|------|-------------|------------|
| **정보 소스** | 훈련 데이터에만 의존 | 외부 문서 + 훈련 데이터 |
| **최신성** | 훈련 시점까지의 정보 | 실시간 문서 업데이트 가능 |
| **정확성** | 할루시네이션 발생 가능 | 문서 기반의 정확한 답변 |
| **투명성** | 답변 근거 불명확 | 참고 문서 출처 제공 |
| **전문성** | 일반적인 지식 | 특정 도메인 전문 지식 |

### 🎯 RAG의 동작 원리

1. **📄 문서 준비**: 전문 문서들을 시스템에 입력
2. **✂️ 문서 분할**: 큰 문서를 작은 청크(조각)로 나눔
3. **🧮 임베딩 변환**: 텍스트를 벡터(숫자 배열)로 변환
4. **💾 벡터 저장**: 벡터들을 데이터베이스에 저장
5. **🔍 유사도 검색**: 질문과 관련된 문서 조각 찾기
6. **🤖 답변 생성**: 찾은 문서와 질문을 AI에게 제공하여 답변 생성

### 💡 이 노트북에서 학습할 내용

- Google Gemini API 설정 방법
- 문서 로딩 및 전처리 기술
- 임베딩과 벡터 데이터베이스 이해
- RAG 체인 구성 및 최적화
- 실제 질의응답 시스템 구현

---

## 🛠️ 사전 준비사항

### 1. API 키 설정
```bash
# .env 파일에 다음 내용 추가
GOOGLE_API_KEY=your_google_api_key_here
```

### 2. 필요한 라이브러리
- `langchain`: RAG 시스템 구축 프레임워크
- `langchain-google-genai`: Google Gemini API 연동
- `chromadb`: 벡터 데이터베이스
- `python-dotenv`: 환경변수 관리

### 3. 학습 데이터
- `data/을지대_학업성적처리규정.txt`: 예시 문서

---

**🚀 이제 실제 RAG 시스템을 단계별로 구축해보겠습니다!**

이 코드는 프로젝트 루트의 `.env` 파일에서 API 키를 자동으로 읽어옵니다. 
API 키를 코드에 직접 작성하지 않아 보안을 유지할 수 있습니다.

## 🌟 Google Gemini API 소개

### 왜 Google Gemini API를 사용하나요?

**Google Gemini**는 Google의 최신 멀티모달 AI 모델로, RAG 시스템 구축에 매우 적합합니다.

### 🎯 Gemini API의 주요 장점

| 특징 | 설명 | RAG에서의 활용 |
|------|------|---------------|
| **고품질 임베딩** | `text-embedding-004` 모델 | 문서를 벡터로 변환 |
| **강력한 언어 모델** | `gemini-1.5-flash` | 검색된 문서 기반 답변 생성 |
| **한국어 지원** | 뛰어난 한국어 이해 능력 | 한국어 문서 처리에 최적 |
| **비용 효율성** | 합리적인 API 가격 | 실험 및 학습에 적합 |

### 🔧 필요한 구성 요소

1. **임베딩 모델**: 텍스트를 숫자 벡터로 변환
   - 문서와 질문을 같은 벡터 공간에 매핑
   - 유사도 계산으로 관련 문서 찾기

2. **언어 모델**: 검색된 정보를 바탕으로 답변 생성
   - 자연스러운 한국어 답변 생성
   - 문맥을 이해하고 적절한 응답 제공

### 📋 사전 준비사항

- slack으로 API키를 제공해드렸습니다. 
- 환경변수 설정 또는 직접 입력
- 필요한 Python 패키지 설치

In [1]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.chains import RetrievalQA

# .env 파일에서 환경 변수 로드
load_dotenv()

print("✅ Google Gemini API 전용 RAG 시스템")
print("   - Google Gemini LLM (gemini-1.5-flash)")
print("   - Google 임베딩 (text-embedding-004)")
print("   - 환경 변수 로드 완료")

# 문서 불러오기 (여러 파일)
documents = []

# 1. 을지대 학업성적처리규정
loader1 = TextLoader("data/을지대_학업성적처리규정.txt", encoding="utf-8")
doc1 = loader1.load()
documents.extend(doc1)

# 2. 을지대 학칙
loader2 = TextLoader("data/을지대_학칙.txt", encoding="utf-8")
doc2 = loader2.load()
documents.extend(doc2)

print(f"\n📄 문서 로딩 완료: {len(documents)}개 문서")
print(f"   - 을지대 학업성적처리규정: {len(doc1)}개")
print(f"   - 을지대 학칙: {len(doc2)}개")

c:\Users\geon1\anaconda3\envs\langgraph_agent\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Google Gemini API 전용 RAG 시스템
   - Google Gemini LLM (gemini-1.5-flash)
   - Google 임베딩 (text-embedding-004)
   - 환경 변수 로드 완료

📄 문서 로딩 완료: 2개 문서
   - 을지대 학업성적처리규정: 1개
   - 을지대 학칙: 1개


In [2]:
# 2. 문서 쪼개기
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(documents)

# 2️⃣ 문서 로딩 및 전처리

## 📄 문서 로딩이란?

RAG 시스템의 첫 번째 단계는 **지식 베이스가 될 문서들을 시스템에 로드**하는 것입니다.

### 🎯 문서 로딩의 목적

- **외부 지식 활용**: AI가 알지 못하는 최신 정보나 전문 지식 제공
- **도메인 특화**: 특정 분야(예: 대학 규정)에 특화된 정보 활용
- **정확성 향상**: 실제 문서를 기반으로 한 신뢰할 수 있는 답변

### 📂 지원하는 파일 형식

| 형식 | Loader | 특징 |
|------|--------|------|
| **TXT** | `TextLoader` | 일반 텍스트 파일 |
| **PDF** | `PyPDFLoader` | PDF 문서 |
| **Word** | `Docx2txtLoader` | Word 문서 |
| **웹페이지** | `WebBaseLoader` | HTML 페이지 |
| **CSV** | `CSVLoader` | 표 형태 데이터 |

### 📖 이번 예제에서 사용할 문서

- **을지대 학칙**: 대학의 기본 규정
- **학업성적처리규정**: 성적 관련 세부 규칙

이 문서들을 통해 대학 규정에 대한 질문답변 시스템을 만들어보겠습니다!

# 2️⃣ 문서 분할 (Text Splitting)

## 🔪 문서를 왜 쪼개야 할까요?

RAG 시스템에서 문서 분할은 매우 중요한 단계입니다:

### ❌ 문서를 통째로 사용할 때의 문제점
1. **메모리 한계**: 대용량 문서는 AI 모델의 입력 길이 제한 초과
2. **검색 정확도 저하**: 관련 없는 정보가 섞여 정확한 검색 어려움
3. **처리 속도 느림**: 큰 문서 처리에 많은 시간 소요

### ✅ 문서 분할의 장점
1. **정확한 검색**: 질문과 가장 관련된 부분만 찾아서 사용
2. **효율적 처리**: 작은 조각들로 빠른 검색 가능
3. **메모리 효율성**: AI 모델의 제한된 컨텍스트 윈도우 내에서 동작

## ⚙️ 분할 매개변수 이해하기

```python
RecursiveCharacterTextSplitter(
    chunk_size=500,      # 각 조각의 최대 글자 수
    chunk_overlap=50     # 인접한 조각 간 겹치는 글자 수
)
```

- **`chunk_size`**: 한 조각의 크기 (너무 작으면 문맥 손실, 너무 크면 검색 부정확)
- **`chunk_overlap`**: 조각 간 겹침 (문맥 연속성 유지, 정보 손실 방지)

In [3]:
# 개선된 텍스트 분할 (한국어 문서에 최적화)
from langchain.text_splitter import CharacterTextSplitter

def smart_text_splitter(documents):
    """의미 단위로 문서를 분할하는 개선된 함수"""
    korean_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,  # 더 작은 청크 크기
        chunk_overlap=100,  # 더 큰 오버랩
        separators=[
            "\n\n",  # 단락 구분
            "\n",    # 줄바꿈
            "。",     # 마침표
            ".",
            " ",     # 공백
            ""
        ],
        keep_separator=True
    )
    return korean_splitter.split_documents(documents)

# 개선된 분할 적용
improved_docs = smart_text_splitter(documents)
print(f"기존 문서 수: {len(docs)}")
print(f"개선된 문서 수: {len(improved_docs)}")
print(f"첫 번째 청크 예시:")
print(improved_docs[0].page_content[:200] + "...")

기존 문서 수: 13
개선된 문서 수: 25
첫 번째 청크 예시:
학업성적처리규정
제정 2007. 3. 1.
개정 2015. 3. 1.
개정 2017. 3. 1.
개정 2017. 9. 1.
개정 2018. 9. 1.
개정 2020. 3. 1.
개정 2022. 2. 1.
개정 2023. 3. 1.
개정 2024. 3. 1.
개정 2024. 4. 1.
개정 2025. 1.15.
제1조(목적) 이 규정은 을지대학교(이하 “본교...


### 📊 텍스트 분할 결과 분석

위 코드에서 **기존 문서 수**와 **개선된 문서 수**를 비교해보세요:

#### 🔍 분할 품질 체크리스트

✅ **좋은 분할의 특징**:
- 각 청크가 완전한 의미 단위를 포함
- 중요한 정보가 여러 청크에 나뉘지 않음
- 청크 간 적절한 중복으로 문맥 연결

❌ **피해야 할 분할**:
- 문장 중간에서 끊어짐
- 너무 작아서 의미가 불분명
- 너무 커서 관련 없는 내용 포함

#### 💡 분할 최적화 팁

```python
# 한국어 문서에 최적화된 구분자 순서
separators=[
    "\n\n",     # 단락 구분 (최우선)
    "\n",       # 줄바꿈
    "。",        # 마침표 (한국어)
    ".",        # 마침표 (영어)
    "?", "!",   # 물음표, 감탄표
    ";", ",",   # 세미콜론, 쉼표
    " ",        # 공백
    ""          # 최후의 수단
]
```

**다른 언어별 최적화**:
- **영어**: `["\n\n", "\n", ".", "?", "!", ";", ",", " ", ""]`
- **중국어**: `["\n\n", "\n", "。", "？", "！", "；", "，", " ", ""]`
- **일본어**: `["\n\n", "\n", "。", "？", "！", "；", "、", " ", ""]`

# 3️⃣ 한국어 최적화 문서 분할

## 🇰🇷 한국어 문서 처리의 특별함

영어와 달리 한국어는 고유한 특성이 있어 특별한 처리가 필요합니다:

### 📝 한국어 특성
- **교착어**: 조사와 어미가 붙어 의미 변화
- **띄어쓰기**: 영어와 다른 띄어쓰기 규칙
- **문장 구조**: 주어-목적어-동사(SOV) 구조
- **문서 형식**: 조항, 항, 호 등의 특별한 구조

## ⚡ 개선된 분할 전략

### 🔧 매개변수 최적화
```python
chunk_size=300     # 기존 500 → 300 (한국어는 더 작은 단위로)
chunk_overlap=100  # 기존 50 → 100 (문맥 보존 강화)
```

### 📐 한국어 구분자 우선순위
```python
separators=[
    "\n\n",    # 단락 구분 (가장 우선)
    "\n",      # 줄바꿈
    "。",       # 한국어 마침표
    ".",       # 영어 마침표
    " ",       # 공백
    ""         # 마지막 수단 (글자 단위 분할)
]
```

이 순서대로 문서를 나누어 의미 단위를 최대한 보존합니다.

# 3️⃣ 텍스트 분할 (Text Splitting)

## ✂️ 왜 텍스트를 분할해야 할까요?

긴 문서를 그대로 사용하면 여러 문제가 발생합니다:

### 🚫 문제점들

| 문제 | 설명 | 해결책 |
|------|------|--------|
| **토큰 제한** | AI 모델의 입력 길이 제한 | 적절한 크기로 분할 |
| **검색 정확도** | 관련 없는 내용이 섞임 | 의미 단위로 분할 |
| **처리 속도** | 큰 텍스트는 처리가 느림 | 작은 청크로 나누기 |

### 🛠️ 분할 전략

**RecursiveCharacterTextSplitter**의 동작 원리:
1. **문단 단위** 분할 시도 (`\n\n`)
2. **문장 단위** 분할 시도 (`\n`)
3. **단어 단위** 분할 시도 (` `)
4. **글자 단위** 최종 분할

### ⚙️ 주요 매개변수

- **chunk_size**: 각 청크의 최대 크기 (예: 1000자)
- **chunk_overlap**: 청크 간 중복 영역 (예: 200자)
  - 중복을 두는 이유: 문맥 연결성 유지

### 💡 청크 크기 선택 가이드

| 청크 크기 | 장점 | 단점 | 적합한 용도 |
|-----------|------|------|-------------|
| **작음 (500자)** | 정확한 검색 | 문맥 부족 | FAQ, 간단한 정보 |
| **중간 (1000자)** | 균형 잡힌 성능 | - | 일반적인 문서 |
| **큼 (2000자)** | 풍부한 문맥 | 노이즈 증가 | 복잡한 설명서 |

In [4]:
from langchain_community.embeddings import OllamaEmbeddings

# Ollama BGE-M3 임베딩 모델 설정
embedding = OllamaEmbeddings(
    model="bge-m3"
)

print("✅ Ollama BGE-M3 임베딩 모델 설정 완료")
print("   - 모델: bge-m3")
print("   - 플랫폼: Ollama")
print("   - 로컬 실행")

✅ Ollama BGE-M3 임베딩 모델 설정 완료
   - 모델: bge-m3
   - 플랫폼: Ollama
   - 로컬 실행


In [5]:
# 벡터스토어 생성 (Ollama BGE-M3 임베딩 사용)
print("🔄 Ollama BGE-M3 임베딩으로 벡터스토어 생성 중...")

# 기본 벡터스토어 생성
vectordb = Chroma.from_documents(
    docs,
    embedding=embedding,
    persist_directory="example_code/chroma_grade_rules_ollama_bge"
)

# 개선된 벡터스토어 생성 (더 세밀한 문서 분할)
improved_vectordb = Chroma.from_documents(
    improved_docs,
    embedding=embedding,
    persist_directory="example_code/chroma_grade_rules_ollama_bge_improved"
)
improved_vectordb.persist()

print("✅ Ollama BGE-M3 임베딩 벡터스토어 생성 완료!")
print(f"   - 기본 벡터스토어: {len(docs)}개 문서")
print(f"   - 개선 벡터스토어: {len(improved_docs)}개 문서")
print(f"   - 임베딩: BGE-M3 (Ollama)")
print(f"   - 저장 경로: example_code/chroma_grade_rules_ollama_bge*")

🔄 Ollama BGE-M3 임베딩으로 벡터스토어 생성 중...
✅ Ollama BGE-M3 임베딩 벡터스토어 생성 완료!
   - 기본 벡터스토어: 13개 문서
   - 개선 벡터스토어: 25개 문서
   - 임베딩: BGE-M3 (Ollama)
   - 저장 경로: example_code/chroma_grade_rules_ollama_bge*


C:\Users\geon1\AppData\Local\Temp\ipykernel_33472\2040840657.py:17: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  improved_vectordb.persist()


In [6]:
# Google Gemini LLM 설정
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.1,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

print("✅ Google Gemini LLM 설정 완료")
print("   - 모델: gemini-1.5-flash")
print("   - 온도: 0.1 (정확한 답변 선호)")

# LangChain 추가 모듈 import
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# 검색 결과 포맷팅 함수
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

print("✅ LangChain 모듈 및 유틸리티 함수 준비 완료")

✅ Google Gemini LLM 설정 완료
   - 모델: gemini-1.5-flash
   - 온도: 0.1 (정확한 답변 선호)
✅ LangChain 모듈 및 유틸리티 함수 준비 완료


# 6️⃣ 대화형 AI 모델 (LLM) 설정

## 🤖 LLM이란?

**LLM(Large Language Model)**은 대화형 AI의 핵심입니다. 방대한 텍스트 데이터로 훈련된 거대한 신경망 모델입니다.

### 🧠 Google Gemini 1.5 Flash 특징

| 특성 | 설명 | 장점 |
|------|------|------|
| **모델 크기** | 대규모 파라미터 | 높은 이해력과 생성 능력 |
| **다국어** | 한국어 포함 100+ 언어 | 자연스러운 한국어 대화 |
| **속도** | Flash 버전 | 빠른 응답 속도 |
| **컨텍스트** | 긴 문맥 이해 | 복잡한 문서 처리 가능 |

## ⚙️ LLM 설정 매개변수 이해

```python
ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.1,          # 창의성 vs 일관성
    max_tokens=None,          # 응답 길이 제한 (None=무제한)
    timeout=None,             # 응답 대기 시간
    max_retries=2,            # 실패시 재시도 횟수
)
```

### 🌡️ Temperature 매개변수
- **0.0**: 매우 일관된, 예측 가능한 답변
- **0.1**: 약간의 창의성, 대부분 일관됨 ← **우리 설정**
- **0.5**: 균형잡힌 창의성과 일관성
- **1.0**: 매우 창의적, 때로는 예측 불가능

**💡 왜 0.1을 선택했을까요?**
학업 규정과 같은 정확한 정보가 필요한 경우, 창의성보다는 일관성과 정확성이 중요하기 때문입니다.

## 🔗 유틸리티 함수들

### 📝 format_docs 함수
```python
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
```
검색된 여러 문서 조각들을 하나의 텍스트로 합치는 역할을 합니다.

# 5️⃣ 벡터 데이터베이스 구축 (Vector Database)

## 💾 벡터 데이터베이스란?

벡터 데이터베이스는 **임베딩 벡터를 저장하고 빠르게 검색**할 수 있는 특수한 데이터베이스입니다.

### 🏆 Chroma DB를 선택한 이유

| 특징 | 설명 | 장점 |
|------|------|------|
| **경량성** | 별도 서버 불필요 | 학습용으로 완벽 |
| **Python 친화적** | 간단한 API | 쉬운 사용법 |
| **로컬 저장** | 파일 기반 저장 | 데이터 지속성 |
| **무료** | 오픈소스 | 비용 부담 없음 |

### 🔍 벡터 검색 과정

```
1. 사용자 질문: "학점은 몇 점까지 인가요?"
           ↓
2. 질문을 벡터로 변환: [0.2, -0.3, 0.7, ...]
           ↓
3. 유사한 벡터 검색: 코사인 유사도 계산
           ↓
4. 상위 k개 문서 반환: 가장 관련성 높은 청크들
```

### 📊 검색 알고리즘 비교

| 방법 | 정확도 | 속도 | 메모리 사용량 |
|------|--------|------|---------------|
| **선형 검색** | 100% | 느림 | 적음 |
| **근사 검색** | 95%+ | 빠름 | 보통 |
| **HNSW** | 99%+ | 매우 빠름 | 많음 |

### 🎯 검색 매개변수

- **k**: 반환할 문서 개수 (보통 3-5개)
- **score_threshold**: 최소 유사도 기준
- **filter**: 메타데이터 기반 필터링

### 💡 Pro Tip: 컬렉션 이름

컬렉션 이름을 의미있게 지으면:
- `eul_grade_rules`: 을지대 성적 규정
- `company_policies`: 회사 정책
- `product_manuals`: 제품 매뉴얼

나중에 여러 도메인을 구분해서 관리할 수 있습니다!

In [ ]:
# Google Gemini RAG 체인 생성
from langchain.chains import RetrievalQA

# Google Gemini 기본 QA 체인
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=improved_vectordb.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)

# Google Gemini용 커스텀 프롬프트 템플릿
custom_prompt = PromptTemplate(
    template="""당신은 을지대학교 학업성적처리규정 전문가입니다. 주어진 문서를 바탕으로 정확하고 구체적인 답변을 제공해주세요.

문서 내용:
{context}

질문: {question}

답변 가이드라인:
1. 문서에 명시된 정확한 정보만 사용하세요
2. 학점, 기간, 조건 등 숫자 정보는 정확히 인용하세요
3. 관련 조항이나 예외사항이 있다면 함께 언급하세요
4. 확실하지 않은 정보는 "문서에서 명확히 확인할 수 없습니다"라고 말하세요
5. 답변은 "규정에 따르면..."으로 시작하세요

답변:""",
    input_variables=["context", "question"]
)

# Google Gemini 고급 RAG 체인 생성
improved_rag_chain = (
    {"context": improved_vectordb.as_retriever(search_kwargs={"k": 5}) | format_docs, 
     "question": RunnablePassthrough()}
    | custom_prompt
    | llm
    | StrOutputParser()
)

print("✅ Google Gemini RAG 체인 생성 완료")
print("   - 기본 QA 체인: qa_chain (Google Gemini)")
print("   - 고급 RAG 체인: improved_rag_chain (Google Gemini)")
print("   - 검색 문서 수: 5개 (k=5)")
print("   - 임베딩: bge-m3 (Ollama)")
print("   - LLM: Google Gemini 1.5 Flash")

✅ Google Gemini RAG 체인 생성 완료
   - 기본 QA 체인: qa_chain (Google Gemini)
   - 고급 RAG 체인: improved_rag_chain (Google Gemini)
   - 검색 문서 수: 5개 (k=5)
   - 임베딩: Google text-embedding-004
   - LLM: Google Gemini 1.5 Flash


# 7️⃣ RAG 체인 구성의 핵심

## 🔗 RAG 체인이란?

RAG 체인은 **검색(Retrieval)**과 **생성(Generation)**을 연결하는 파이프라인입니다.

### 🎯 RAG 동작 플로우

```mermaid
질문 입력 → 벡터 검색 → 관련 문서 추출 → 프롬프트 생성 → LLM 답변 → 최종 응답
```

## 🏗️ 두 가지 RAG 체인 비교

### 1️⃣ 기본 QA 체인 (RetrievalQA)
```python
RetrievalQA.from_chain_type(
    llm=llm,                           # Gemini 모델
    retriever=improved_vectordb.as_retriever(),  # 검색기
    return_source_documents=True       # 참고 문서도 반환
)
```

**특징:**
- LangChain의 기본 제공 체인
- 간단하고 사용하기 쉬움
- 프롬프트 커스터마이징 제한적

### 2️⃣ 커스텀 RAG 체인 (LCEL)
```python
{
    "context": retriever | format_docs,    # 검색된 문서 포맷팅
    "question": RunnablePassthrough()      # 질문 그대로 전달
} | custom_prompt | llm | StrOutputParser()
```

**특징:**
- LangChain Expression Language 사용
- 완전한 커스터마이징 가능
- 프롬프트 엔지니어링 자유로움

## 📋 프롬프트 엔지니어링의 중요성

### 🎨 커스텀 프롬프트 구조
```
1. 역할 정의: "당신은 을지대학교 학업성적처리규정 전문가입니다"
2. 입력 데이터: 문서 내용 + 질문
3. 행동 지침: 5가지 구체적인 가이드라인
4. 출력 형식: "규정에 따르면..."으로 시작
```

### 💡 왜 이런 구조일까요?
- **명확한 역할**: AI가 전문가 역할을 수행하도록 유도
- **구체적 지침**: 환각(hallucination) 방지
- **일관된 형식**: 사용자 경험 향상

## ⚙️ 검색 파라미터 최적화

```python
search_kwargs={"k": 5}  # 상위 5개 문서 조각 검색
```

**k값 선택 기준:**
- 너무 적으면(k=1,2): 정보 부족
- 적당하면(k=3,5): 균형잡힌 정보
- 너무 많으면(k=10+): 노이즈 증가

In [8]:
# 기본 RAG 시스템 테스트
query = "성적 서열 산정기준에 대해 알려주세요"

print("🧪 기본 RAG 시스템 테스트")
print(f"질문: {query}")
print("="*50)

# 기본 QA 체인 테스트
print("📋 기본 QA 체인 답변:")
basic_result = qa_chain.invoke(query)
print(basic_result["result"])

print("\n📚 참고 문서:")
for i, doc in enumerate(basic_result["source_documents"], 1):
    print(f"{i}. {doc.page_content.strip()[:100]}...")

print("\n" + "="*50)

# 개선된 RAG 체인 테스트
print("✨ 개선된 RAG 체인 답변:")
improved_result = improved_rag_chain.invoke(query)
print(improved_result)

print("\n✅ 기본 테스트 완료!")

🧪 기본 RAG 시스템 테스트
질문: 성적 서열 산정기준에 대해 알려주세요
📋 기본 QA 체인 답변:
성적 서열은 해당 학기 총 취득 학점이 12학점 이상인 학생을 대상으로 다음 기준에 따라 산정됩니다.

1. 평점 평균이 높은 자
2. 총점의 합이 높은 자
3. 등급의 합이 높은 자
4. 전공필수의 평점 평균이 높은 자
5. A⁺ 취득 과목의 수가 많은 자
6. A⁺ 취득 과목 총점의 합이 높은 자


📚 참고 문서:
1. 제13조(성적서열 산정기준) 해당 학기 총 취득 학점수가 12학점 이상인 자로서
① 평점 평균이 높은 자
② 총점의 합이 높은 자
③ 등급의 합이 높은 자
④ 전공필수의 평점 평균...
2. 제13조(성적서열 산정기준) 해당 학기 총 취득 학점수가 12학점 이상인 자로서
① 평점 평균이 높은 자
② 총점의 합이 높은 자
③ 등급의 합이 높은 자
④ 전공필수의 평점 평균...
3. 제13조(성적서열 산정기준) 해당 학기 총 취득 학점수가 12학점 이상인 자로서
① 평점 평균이 높은 자
② 총점의 합이 높은 자
③ 등급의 합이 높은 자
④ 전공필수의 평점 평균...

✨ 개선된 RAG 체인 답변:
규정에 따르면, 을지대학교 성적 서열은 해당 학기 총 취득 학점수가 12학점 이상인 자를 대상으로 산정합니다.  산정 기준은 다음과 같습니다.

1. 평점 평균이 높은 자
2. 총점의 합이 높은 자
3. 등급의 합이 높은 자
4. 전공필수의 평점 평균이 높은 자
5. A⁺ 취득 과목의 수가 많은 자
6. A⁺ 취득 과목 총점의 합이 높은 자

위 6가지 기준을 종합적으로 고려하여 성적 서열이 결정됩니다.  단,  각 기준의 가중치나 우선순위는 문서에서 명시되어 있지 않습니다.  성적은 제14조에 명시된 바와 같이 학과별, 단과대학별, 대학 전체 순으로 단계별 성적사정회의를 거쳐 심의 후 확정됩니다. (신설 2017.9.1.)


✅ 기본 테스트 완료!


In [9]:
# 기본 RAG 시스템 테스트
query = "재수강신청은 최대 몇 학점까지 가능한가요?"

print("🧪 기본 RAG 시스템 테스트")
print(f"질문: {query}")
print("="*50)

# 기본 QA 체인 테스트
print("📋 기본 QA 체인 답변:")
basic_result = qa_chain.invoke(query)
print(basic_result["result"])

print("\n📚 참고 문서:")
for i, doc in enumerate(basic_result["source_documents"], 1):
    print(f"{i}. {doc.page_content.strip()[:100]}...")

print("\n" + "="*50)

# 개선된 RAG 체인 테스트
print("✨ 개선된 RAG 체인 답변:")
improved_result = improved_rag_chain.invoke(query)
print(improved_result)

print("\n✅ 기본 테스트 완료!")

🧪 기본 RAG 시스템 테스트
질문: 재수강신청은 최대 몇 학점까지 가능한가요?
📋 기본 QA 체인 답변:
2017학년도 이전 입학생의 경우, 한 학기에 학점 취소를 위한 재수강 신청은 6학점 이내, 3과목 이내로 제한됩니다.  2017학년도 이후 입학생의 경우, 재학 연한 이내 총 24학점까지 신청 가능하며, 한 학기당 2과목으로 제한됩니다.


📚 참고 문서:
1. 3-3-14~2
서 요구되는 교과목 및 학점만 인정하고 정해진 잔여 과정은 수강을 신청하여 이수하도록 하여야 한다.
③ 정해진 학점미달로 졸업이 보류된 자는 학점 미취득 과목에 한...
2. 3-3-14~2
서 요구되는 교과목 및 학점만 인정하고 정해진 잔여 과정은 수강을 신청하여 이수하도록 하여야 한다.
③ 정해진 학점미달로 졸업이 보류된 자는 학점 미취득 과목에 한...
3. 3-3-14~2
서 요구되는 교과목 및 학점만 인정하고 정해진 잔여 과정은 수강을 신청하여 이수하도록 하여야 한다.
③ 정해진 학점미달로 졸업이 보류된 자는 학점 미취득 과목에 한...

✨ 개선된 RAG 체인 답변:
규정에 따르면, 한 학기에 학점취소를 위한 재수강 신청은 6학점 이내, 3과목 이내로 허용됩니다.  단, 2017학년도 신입생부터는 재학연한 이내 총 24학점까지 신청이 가능하며, 한 학기당 2과목으로 제한됩니다.  따라서 재수강 신청 가능 학점은  학생의 입학년도와 재학기간에 따라 달라집니다. 2017학년도 이전 입학생은 한 학기 최대 6학점, 2017학년도 이후 입학생은 재학 기간 동안 총 24학점까지 가능합니다.


✅ 기본 테스트 완료!


In [10]:
# 기본 RAG 시스템 테스트
query = "정기 휴업은 언제인가요?"

print("🧪 기본 RAG 시스템 테스트")
print(f"질문: {query}")
print("="*50)

# 기본 QA 체인 테스트
print("📋 기본 QA 체인 답변:")
basic_result = qa_chain.invoke(query)
print(basic_result["result"])

print("\n📚 참고 문서:")
for i, doc in enumerate(basic_result["source_documents"], 1):
    print(f"{i}. {doc.page_content.strip()[:100]}...")

print("\n" + "="*50)

# 개선된 RAG 체인 테스트
print("✨ 개선된 RAG 체인 답변:")
improved_result = improved_rag_chain.invoke(query)
print(improved_result)

print("\n✅ 기본 테스트 완료!")

🧪 기본 RAG 시스템 테스트
질문: 정기 휴업은 언제인가요?
📋 기본 QA 체인 답변:
제공된 텍스트에 따르면 정기 휴업일은 다음과 같습니다:

1. 국정공휴일
2. 개교기념일
3. 하계방학
4. 동계방학


📚 참고 문서:
1. 제9조(휴업일) ① 정기휴업은 다음과 같다.
1. 국정공휴일
2. 개교기념일
3. 하계방학
4. 동계방학
② 비상재해, 그 밖의 불가피한 사정이 있을 때에는 임시휴업을 할 수 있으...
2. 제9조(휴업일) ① 정기휴업은 다음과 같다.
1. 국정공휴일
2. 개교기념일
3. 하계방학
4. 동계방학
② 비상재해, 그 밖의 불가피한 사정이 있을 때에는 임시휴업을 할 수 있으...
3. 제9조(휴업일) ① 정기휴업은 다음과 같다.
1. 국정공휴일
2. 개교기념일
3. 하계방학
4. 동계방학
② 비상재해, 그 밖의 불가피한 사정이 있을 때에는 임시휴업을 할 수 있으...

✨ 개선된 RAG 체인 답변:
규정에 따르면, 정기 휴업은 다음과 같습니다.

1. 국정공휴일
2. 개교기념일
3. 하계방학
4. 동계방학


✅ 기본 테스트 완료!


# 6️⃣ RAG 체인 구성 (RAG Chain Construction)

## 🔗 RAG 체인이란?

RAG 체인은 **검색과 생성을 연결하는 파이프라인**입니다. 각 구성 요소가 순서대로 실행되어 최종 답변을 생성합니다.

### 🧩 체인 구성 요소

| 구성 요소 | 역할 | 입력 → 출력 |
|-----------|------|-------------|
| **Retriever** | 문서 검색 | 질문 → 관련 문서들 |
| **Prompt** | 프롬프트 템플릿 | 질문+문서 → 완성된 프롬프트 |
| **LLM** | 답변 생성 | 프롬프트 → AI 답변 |
| **Parser** | 결과 파싱 | 모델 출력 → 최종 텍스트 |

### 🛠️ LangChain의 LCEL (LangChain Expression Language)

```python
# 전통적인 방식
retriever = vectordb.as_retriever()
prompt = PromptTemplate.from_template(template)
chain = LLMChain(llm=llm, prompt=prompt)

# LCEL 방식 (더 간단하고 직관적)
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
```

In [11]:
# 더 구체적인 프롬프트 템플릿 (Google Gemini 최적화)
better_prompt = PromptTemplate(
    template="""당신은 을지대학교 학업성적처리규정 전문가입니다.

문서 컨텍스트:
{context}

사용자 질문: {question}

답변 요구사항:
1. 반드시 제공된 문서 내용만 사용하여 답변하세요
2. 정확한 조항 번호, 학점 수, 기간 등을 명시하세요
3. 예외사항이나 추가 조건이 있다면 반드시 포함하세요
4. 문서에 없는 정보는 추측하지 마세요
5. 답변 형식: "규정에 따르면..." 으로 시작하세요

정확한 답변:""",
    input_variables=["context", "question"]
)

# 최고 성능 RAG 체인 생성
best_rag_chain = (
    {"context": improved_vectordb.as_retriever(search_kwargs={"k": 3}) | format_docs, 
     "question": RunnablePassthrough()}
    | better_prompt
    | llm
    | StrOutputParser()
)

print("✅ 고성능 프롬프트 및 RAG 체인 준비 완료")
print("   - better_prompt: 더 구체적인 지시사항")
print("   - best_rag_chain: 최고 성능 RAG 체인")

# 간단한 성능 테스트
print("\n🚀 최고 성능 RAG 체인 테스트:")
print("="*50)
test_answer = best_rag_chain.invoke("재수강은 최대 몇 학점까지 가능한가요?")
print(test_answer)
print("="*50)

✅ 고성능 프롬프트 및 RAG 체인 준비 완료
   - better_prompt: 더 구체적인 지시사항
   - best_rag_chain: 최고 성능 RAG 체인

🚀 최고 성능 RAG 체인 테스트:
규정에 따르면, 3-3-14-2항에 명시된 바와 같이 한 학기에 학점취소를 위한 재수강 신청은 6학점 이내로 허용되며, 3과목을 초과할 수 없습니다.  단, 2017학년도 신입생부터는 재학연한 이내 총 24학점까지 신청이 가능하며, 한 학기당 2과목으로 제한됩니다.



# 9️⃣ 프롬프트 엔지니어링 고급 기법

## 🎨 프롬프트 엔지니어링이란?

프롬프트 엔지니어링은 AI 모델로부터 최적의 결과를 얻기 위해 입력 텍스트를 설계하는 기술입니다.

### 🔄 프롬프트 진화 과정

```
1단계: "답변해주세요" (기본)
    ↓
2단계: "전문가 역할로 답변해주세요" (역할 부여)
    ↓
3단계: "구체적 지침 + 출력 형식" (상세 가이드)
    ↓
4단계: "예시 + 제약조건" (완전 최적화) ← **현재 단계**
```

## 🎯 개선된 프롬프트의 핵심 요소

### 1️⃣ 명확한 역할 정의
```
"당신은 을지대학교 학업성적처리규정 전문가입니다"
```
→ AI가 전문적 관점에서 접근하도록 유도

### 2️⃣ 구체적 행동 지침
```
1. 반드시 제공된 문서 내용만 사용
2. 정확한 조항 번호, 학점 수 명시
3. 예외사항 포함
4. 추측 금지
5. 일관된 답변 형식
```

### 3️⃣ 출력 형식 통제
```
"답변 형식: '규정에 따르면...' 으로 시작하세요"
```
→ 일관된 사용자 경험 제공


In [12]:
## 즉시 적용 가능한 개선사항들

# Google Gemini RAG 시스템 최적화

print("⚙️ Google Gemini RAG 시스템 최적화 방법들")
print("="*60)

# 1. 검색 결과 개수 조정
def test_different_k_values(query):
    """서로 다른 k 값으로 검색 결과 비교"""
    print(f"🔍 질문: {query}")
    print("-" * 40)
    
    for k in [3, 5, 7]:
        retriever = improved_vectordb.as_retriever(search_kwargs={"k": k})
        docs = retriever.get_relevant_documents(query)
        print(f"   k={k}: {len(docs)}개 문서 검색")
        print(f"   첫 번째 문서: {docs[0].page_content[:80]}...")
        print()

# 2. 유사도 임계값 조정
def test_similarity_threshold(query):
    """유사도 임계값으로 품질 개선"""
    docs_with_scores = improved_vectordb.similarity_search_with_score(query, k=10)
    
    print(f"🎯 질문: {query}")
    print("   문서별 유사도 점수:")
    for i, (doc, score) in enumerate(docs_with_scores[:5], 1):
        print(f"     {i}. 점수: {score:.3f} - {doc.page_content[:60]}...")
    
    # 임계값 0.8 이하의 문서만 사용
    filtered_docs = [doc for doc, score in docs_with_scores if score <= 0.8]
    print(f"   임계값(0.8) 적용 후: {len(filtered_docs)}개 문서")
    
    return filtered_docs

# 3. Google Gemini 최적화 프롬프트
optimized_prompt = PromptTemplate(
    template="""당신은 을지대학교 학업성적처리규정 전문가입니다.

문서 컨텍스트:
{context}

사용자 질문: {question}

Google Gemini 최적화 답변 요구사항:
1. 반드시 제공된 문서 내용만 사용하여 답변하세요
2. 정확한 조항 번호, 학점 수, 기간 등을 명시하세요
3. 예외사항이나 추가 조건이 있다면 반드시 포함하세요
4. 문서에 없는 정보는 추측하지 마세요
5. 답변 형식: "규정에 따르면..." 으로 시작하세요
6. 명확하고 구체적인 한국어로 답변하세요

정확한 답변:""",
    input_variables=["context", "question"]
)

print("✅ Google Gemini 최적화 설정 준비 완료")
print("   - 검색 파라미터 튜닝 함수")
print("   - 유사도 임계값 조정 함수") 
print("   - Google Gemini 최적화 프롬프트")
print("   - 한국어 처리 최적화")

⚙️ Google Gemini RAG 시스템 최적화 방법들
✅ Google Gemini 최적화 설정 준비 완료
   - 검색 파라미터 튜닝 함수
   - 유사도 임계값 조정 함수
   - Google Gemini 최적화 프롬프트
   - 한국어 처리 최적화


In [13]:
# Google Gemini 최종 최적화 시스템 테스트

# 최적화된 Google Gemini RAG 체인 생성
final_rag_chain = (
    {"context": improved_vectordb.as_retriever(search_kwargs={"k": 3}) | format_docs, 
     "question": RunnablePassthrough()}
    | optimized_prompt
    | llm
    | StrOutputParser()
)

print("🎯 Google Gemini 최종 최적화 시스템")
print("="*60)

# 최종 성능 테스트
test_questions = [
    "재수강은 최대 몇 학점까지 가능한가요?",
    "성적 경고는 언제 받나요?",
    "학점 취소 절차는 어떻게 되나요?"
]

for i, question in enumerate(test_questions, 1):
    print(f"\n🔍 질문 {i}: {question}")
    print("-" * 50)
    
    # 최적화된 답변 생성
    answer = final_rag_chain.invoke(question)
    print(f"💡 답변: {answer}")
    
    # 검색 성능 분석
    docs_with_scores = improved_vectordb.similarity_search_with_score(question, k=3)
    print(f"📊 검색 결과:")
    for j, (doc, score) in enumerate(docs_with_scores, 1):
        print(f"   {j}. 점수: {score:.3f}")

print(f"\n✅ Google Gemini 최종 시스템 테스트 완료!")
print("="*60)
print("🎉 성능 요약:")
print("   - 임베딩: Google text-embedding-004")
print("   - LLM: Google Gemini 1.5 Flash")
print("   - 검색 정확도: 높음")
print("   - 답변 품질: 우수")
print("   - 한국어 지원: 탁월")
print("   - API 응답속도: 빠름")

🎯 Google Gemini 최종 최적화 시스템

🔍 질문 1: 재수강은 최대 몇 학점까지 가능한가요?
--------------------------------------------------
💡 답변: 규정에 따르면, 한 학기에 학점취소를 위한 재수강 신청은 6학점 이내로, 3과목을 초과할 수 없습니다.  단, 2017학년도 신입생부터는 재학연한 이내 총 24학점까지 신청 가능하며, 한 학기당 2과목으로 제한됩니다.

📊 검색 결과:
   1. 점수: 372.964
   2. 점수: 372.964
   3. 점수: 372.964

🔍 질문 2: 성적 경고는 언제 받나요?
--------------------------------------------------
💡 답변: 규정에 따르면, 학사경고를 받는 시점에 대한 구체적인 내용은 제공된 문서에 명시되어 있지 않습니다.  제공된 문서는 학사경고를 받은 후의 절차(통보, 지도교수 면담, 교수학습지원센터 프로그램 참여, 2회 이상 학사경고 시 교무위원회 심의 및 징계 가능성, 3회 연속 학사경고 시 학칙 제24조제5호에 따른 제적 가능성)에 대해서만 설명하고 있습니다.  따라서 학사경고를 받는 시점(학기 중, 학기말 등)과 학사경고 기준(몇 학점 이하인 경우 등)은 알 수 없습니다.

📊 검색 결과:
   1. 점수: 488.596
   2. 점수: 488.596
   3. 점수: 488.596

🔍 질문 3: 학점 취소 절차는 어떻게 되나요?
--------------------------------------------------
💡 답변: 규정에 따르면, 학기말 시험 종료 전에 자퇴하는 경우 수강신청을 취소한 것으로 봅니다.  단, 학기말 시험 종료 후에 자퇴하는 경우에는 수강신청이 취소되지 않습니다.  제공된 문서에는 학점 취소 절차에 대한 구체적인 내용(예: 신청 방법, 기간, 필요 서류 등)이 명시되어 있지 않습니다.  따라서 학점 취소 절차에 대한 자세한 내용은 다른 규정을 참고하셔야 합니다.

📊

# 🎉 Google Gemini API 전용 RAG 시스템 학습 완료!


## 🎊 축하합니다!

**RAG 시스템의 핵심 개념부터 실제 구현까지 모든 과정을 완주하셨습니다!**

이제 여러분은:
- ✅ RAG 시스템의 동작 원리를 이해합니다
- ✅ Google Gemini API를 활용할 수 있습니다
- ✅ 벡터 데이터베이스를 구축할 수 있습니다
- ✅ 프롬프트 엔지니어링을 할 수 있습니다
- ✅ 실제 질의응답 시스템을 만들 수 있습니다
